<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/doc2vecaozora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc


fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1
を参照

In [37]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [38]:
import zipfile
import os.path
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument

mecab=MeCab.Tagger()
mecab.parse("")

list = [
      {"auther":{
          "name":"宮沢 賢治",
          "url":"https://www.aozora.gr.jp/cards/000081/files/"},
       "books":[
                {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
                {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
                {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
                {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
                {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
       ]},
      {"auther":{
          "name":"芥川 龍之介",
          "url":"https://www.aozora.gr.jp/cards/000879/files/"},
       "books":[
                {"name":"羅生門","zipname":"127_ruby_150.zip"},
                {"name":"鼻","zipname":"42_ruby_154.zip"},
                {"name":"河童","zipname":"69_ruby_1321.zip"},
                {"name":"歯車","zipname":"42377_ruby_34744.zip"},
                {"name":"老年","zipname":"131_ruby_241.zip"},
       ]},
       {"auther":{
          "name":"太宰 治",
          "url":"https://www.aozora.gr.jp/cards/000035/files/"},
       "books":[
                {"name":"斜陽","zipname":"1565_ruby_8220.zip"},
                {"name":"走れメロス","zipname":"1567_ruby_4948.zip"},
                {"name":"津軽","zipname":"2282_ruby_1996.zip"},
                {"name":"お伽草紙","zipname":"307_ruby_3042.zip"},
                {"name":"人間失格","zipname":"301_ruby_5915.zip"},
       ]},
       {"auther":{
          "name":"夏目 漱石",
          "url":"https://www.aozora.gr.jp/cards/000148/files/"},
       "books":[
                {"name":"吾輩は猫である","zipname":"789_ruby_5639.zip"},
                {"name":"坊っちゃん","zipname":"752_ruby_2438.zip"},
                {"name":"草枕","zipname":"776_ruby_6020.zip"},
                {"name":"虞美人草","zipname":"761_ruby_1861.zip"},
                {"name":"三四郎","zipname":"794_ruby_4237.zip"},
       ]},
      
]

def book_list():
  for novelist in list:
    auther = novelist["auther"]
    for book in novelist["books"]:
      yield auther, book

def read_book(auther, book):
  zipname = book["zipname"]
  if not os.path.exists(zipname):
    req.urlretrieve(auther["url"] + zipname, zipname)
  zipname = book["zipname"]
  with zipfile.ZipFile(zipname,"r") as zf:
    for filename in zf.namelist():
      with zf.open(filename,"r") as f:
        return f.read().decode("shift-jis") 

def split_words(text):
  node = mecab.parseToNode(text)
  wakati_words = []
  while node is not None:
    hinshi = node.feature.split(",")[0]
    if hinshi in ["名詞"]:
      wakati_words.append(node.surface)
    elif hinshi in ["動詞","形容詞"]:
      wakati_words.append(node.feature.split(",")[6])
    node = node.next
  return wakati_words

documents = []

for auther, book in book_list():
  words = read_book(auther, book)
  wakati_words = split_words(words)
  document = TaggedDocument(
     wakati_words, [auther["name"] + ":" + book["name"]]
     )
  documents.append(document)

model = models.Doc2Vec(
    documents, dm=1, vector_size=300, window=5, min_count=1)

model.save('aozora.model')

print("モデル作成完了")

モデル作成完了


In [39]:
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models


